In [1]:
import os
import dlt
from oss_analytics.source.github_source import paginated_getter
from dotenv import load_dotenv

load_dotenv()
github_token = os.getenv("GITHUB_TOKEN") or ""

In [2]:
pipeline = dlt.pipeline(
        pipeline_name="github_pipeline",
        destination="duckdb",
        dataset_name="events_dataset",
    )

run_info = pipeline.run(
    paginated_getter(
        repo="supabase/supabase",
        endpoint="events",
        token=github_token
    ),
    table_name="events",
    write_disposition="replace",
)

2025-06-19 16:16:02,403|[WARNING]|33588|8403695680|dlt|validate.py|verify_normalized_table:48|In schema `github`: The column payload__pull_request__assignee in table events did not receive any data during this load. Therefore, its type couldn't be inferred. Unless a type hint is provided, the column will not be materialized in the destination. One way to provide a type hint is to use the 'columns' argument in the '@dlt.resource' decorator. For example:

@dlt.resource(columns={'payload__pull_request__assignee': {'data_type': 'text'}})


2025-06-19 16:16:02,404|[WARNING]|33588|8403695680|dlt|validate.py|verify_normalized_table:48|In schema `github`: The column payload__pull_request__milestone in table events did not receive any data during this load. Therefore, its type couldn't be inferred. Unless a type hint is provided, the column will not be materialized in the destination. One way to provide a type hint is to use the 'columns' argument in the '@dlt.resource' decorator. For example:


### Database inspections
- table_names
- table df
- pipeline.sql_client()
- dlt pipeline 
  - _dlt_pipeline_state
  - _dlt_loads
  - _dlt_version
- DuckDB
  - list of tables: FROM information_schema.tables
  - df: FROM events_dataset.events

In [5]:
print(pipeline.dataset(dataset_type="default").schema.data_table_names())

['events', 'events__payload__commits', 'events__payload__pull_request__head__repo__topics', 'events__payload__pull_request__base__repo__topics', 'events__payload__comment__performed_via_github_app__events', 'events__payload__issue__labels', 'events__payload__issue__assignees', 'events__payload__pull_request__assignees', 'events__payload__pull_request__requested_reviewers', 'events__payload__pull_request__labels', 'events__payload__release__mentions', 'events__payload__pull_request__requested_teams']


In [7]:
pipeline.dataset(dataset_type="default").events.df().head()

,id,type,actor__id,actor__login,actor__display_login,actor__gravatar_id,actor__url,actor__avatar_url,repo__id,repo__name,...,payload__comment__performed_via_github_app__permissions__attestations,payload__comment__performed_via_github_app__permissions__discussions,payload__comment__performed_via_github_app__permissions__merge_queues,payload__comment__performed_via_github_app__permissions__models,payload__comment__performed_via_github_app__permissions__packages,payload__comment__performed_via_github_app__permissions__pages,payload__comment__performed_via_github_app__permissions__repository_projects,payload__comment__performed_via_github_app__permissions__security_events,payload__comment__performed_via_github_app__permissions__vulnerability_alerts,payload__issue__pull_request__merged_at
0,51094393247,PushEvent,19742402,joshenlim,joshenlim,,https://api.github.com/users/joshenlim,https://avatars.githubusercontent.com/u/19742402?,214587193,supabase/supabase,...,None,None,None,None,None,None,None,None,None,NaT
1,51094317768,PushEvent,37541088,jordienr,jordienr,,https://api.github.com/users/jordienr,https://avatars.githubusercontent.com/u/37541088?,214587193,supabase/supabase,...,None,None,None,None,None,None,None,None,None,NaT
2,51094089084,WatchEvent,22812353,hktklxz,hktklxz,,https://api.github.com/users/hktklxz,https://avatars.githubusercontent.com/u/22812353?,214587193,supabase/supabase,...,None,None,None,None,None,None,None,None,None,NaT
3,51093887562,IssueCommentEvent,35613825,vercel[bot],vercel,,https://api.github.com/users/vercel[bot],https://avatars.githubusercontent.com/u/35613825?,214587193,supabase/supabase,...,None,None,None,None,None,None,None,None,None,NaT
4,51093881002,PullRequestEvent,19742402,joshenlim,joshenlim,,https://api.github.com/users/joshenlim,https://avatars.githubusercontent.com/u/19742402?,214587193,supabase/supabase,...,None,None,None,None,None,None,None,None,None,NaT


In [8]:
pipeline.dataset(dataset_type="default").events.df().shape

(278, 757)

In [ ]:
sql = """
SELECT *
FROM events e
JOIN events__payload__pull_request__base__repo__topics c
ON e._dlt_id = c._dlt_parent_id
"""
with pipeline.sql_client() as client:
    with client.execute_query(sql) as cursor:
        data = cursor.df()
data

,name
0,_dlt_loads
1,_dlt_pipeline_state
2,_dlt_version
3,events
4,events__payload__comment__performed_via_github...
5,events__payload__commits
6,events__payload__issue__assignees
7,events__payload__issue__labels
8,events__payload__pull_request__assignees
9,events__payload__pull_request__base__repo__topics


In [10]:
pipeline.dataset(dataset_type="default")._dlt_pipeline_state.df()

,version,engine_version,pipeline_name,state,created_at,version_hash,_dlt_load_id,_dlt_id
0,1,4,github_pipeline,eNpdj1FrwkAQhP/LvhrEghoM+KDUUrDqa2spxyXZmmsvl5...,2025-06-19 09:12:48.066185+00:00,6MOXBOD7z7mLw9Y5Ke+Mljo7vEEymZrFNwHx50AcK6A=,1750324361.488451,N/87pgUi4lLHIg


In [11]:
pipeline.dataset(dataset_type="default")._dlt_loads.df()

,load_id,schema_name,status,inserted_at,schema_version_hash
0,1750324361.488451,github,0,2025-06-19 09:12:49.604073+00:00,GmYMyAjUwhFJbnbfHxlD5mCToZNSZDQrL7vN3aqkAv0=
1,1750324555.113997,github,0,2025-06-19 09:16:03.746041+00:00,K4L2LSS1hk8vJA3SjtQQzA84Etv2zyy597DDjG1viaM=


In [12]:
pipeline.dataset(dataset_type="default")._dlt_version.df()

,version,engine_version,inserted_at,schema_name,version_hash,schema
0,2,11,2025-06-19 09:12:48.378930+00:00,github,GmYMyAjUwhFJbnbfHxlD5mCToZNSZDQrL7vN3aqkAv0=,"{""version"":2,""version_hash"":""GmYMyAjUwhFJbnbfH..."
1,3,11,2025-06-19 09:16:02.581367+00:00,github,K4L2LSS1hk8vJA3SjtQQzA84Etv2zyy597DDjG1viaM=,"{""version"":3,""version_hash"":""K4L2LSS1hk8vJA3Sj..."


In [41]:
import duckdb

# Connect to your database file
conn = duckdb.connect('github_pipeline.duckdb')

sql = """
SELECT *
FROM information_schema.tables
"""
conn.execute(sql).df()

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,github_pipeline,events_dataset,events,BASE TABLE,None,None,None,None,None,YES,NO,None,None
1,github_pipeline,events_dataset,events__payload__comment__performed_via_github...,BASE TABLE,None,None,None,None,None,YES,NO,None,None
2,github_pipeline,events_dataset,events__payload__commits,BASE TABLE,None,None,None,None,None,YES,NO,None,None
3,github_pipeline,events_dataset,events__payload__issue__assignees,BASE TABLE,None,None,None,None,None,YES,NO,None,None
4,github_pipeline,events_dataset,events__payload__issue__labels,BASE TABLE,None,None,None,None,None,YES,NO,None,None
5,github_pipeline,events_dataset,events__payload__pull_request__assignees,BASE TABLE,None,None,None,None,None,YES,NO,None,None
6,github_pipeline,events_dataset,events__payload__pull_request__base__repo__topics,BASE TABLE,None,None,None,None,None,YES,NO,None,None
7,github_pipeline,events_dataset,events__payload__pull_request__head__repo__topics,BASE TABLE,None,None,None,None,None,YES,NO,None,None
8,github_pipeline,events_dataset,events__payload__pull_request__labels,BASE TABLE,None,None,None,None,None,YES,NO,None,None
9,github_pipeline,events_dataset,events__payload__pull_request__requested_revie...,BASE TABLE,None,None,None,None,None,YES,NO,None,None


In [ ]:
sql = """
SELECT *
FROM events_dataset.events
"""
conn.execute(sql).df()

,id,type,actor__id,actor__login,actor__display_login,actor__gravatar_id,actor__url,actor__avatar_url,repo__id,repo__name,...,payload__comment__performed_via_github_app__permissions__attestations,payload__comment__performed_via_github_app__permissions__discussions,payload__comment__performed_via_github_app__permissions__merge_queues,payload__comment__performed_via_github_app__permissions__models,payload__comment__performed_via_github_app__permissions__packages,payload__comment__performed_via_github_app__permissions__pages,payload__comment__performed_via_github_app__permissions__repository_projects,payload__comment__performed_via_github_app__permissions__security_events,payload__comment__performed_via_github_app__permissions__vulnerability_alerts,payload__issue__pull_request__merged_at
0,51094393247,PushEvent,19742402,joshenlim,joshenlim,,https://api.github.com/users/joshenlim,https://avatars.githubusercontent.com/u/19742402?,214587193,supabase/supabase,...,None,None,None,None,None,None,None,None,None,NaT
1,51094317768,PushEvent,37541088,jordienr,jordienr,,https://api.github.com/users/jordienr,https://avatars.githubusercontent.com/u/37541088?,214587193,supabase/supabase,...,None,None,None,None,None,None,None,None,None,NaT
2,51094089084,WatchEvent,22812353,hktklxz,hktklxz,,https://api.github.com/users/hktklxz,https://avatars.githubusercontent.com/u/22812353?,214587193,supabase/supabase,...,None,None,None,None,None,None,None,None,None,NaT
3,51093887562,IssueCommentEvent,35613825,vercel[bot],vercel,,https://api.github.com/users/vercel[bot],https://avatars.githubusercontent.com/u/35613825?,214587193,supabase/supabase,...,None,None,None,None,None,None,None,None,None,NaT
4,51093881002,PullRequestEvent,19742402,joshenlim,joshenlim,,https://api.github.com/users/joshenlim,https://avatars.githubusercontent.com/u/19742402?,214587193,supabase/supabase,...,None,None,None,None,None,None,None,None,None,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,50995668118,PullRequestReviewEvent,22714384,Ziinc,Ziinc,,https://api.github.com/users/Ziinc,https://avatars.githubusercontent.com/u/22714384?,214587193,supabase/supabase,...,None,None,None,None,None,None,None,None,None,NaT
274,50995566397,PullRequestReviewEvent,37541088,jordienr,jordienr,,https://api.github.com/users/jordienr,https://avatars.githubusercontent.com/u/37541088?,214587193,supabase/supabase,...,None,None,None,None,None,None,None,None,None,NaT
275,50994858024,PushEvent,568291,ivasilov,ivasilov,,https://api.github.com/users/ivasilov,https://avatars.githubusercontent.com/u/568291?,214587193,supabase/supabase,...,None,None,None,None,None,None,None,None,None,NaT
276,50994857827,DeleteEvent,568291,ivasilov,ivasilov,,https://api.github.com/users/ivasilov,https://avatars.githubusercontent.com/u/568291?,214587193,supabase/supabase,...,None,None,None,None,None,None,None,None,None,NaT


## a